In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os

# set env flags to catch BLAS used for scipy/numpy 
# to only use 1 cpu, n_cpus will be totally controlled by csky
if False:
    os.environ['MKL_NUM_THREADS'] = "1"
    os.environ['NUMEXPR_NUM_THREADS'] = "1"
    os.environ['OMP_NUM_THREADS'] = "1"
    os.environ['OPENBLAS_NUM_THREADS'] = "1"
    os.environ['VECLIB_MAXIMUM_THREADS'] = "1"

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'w'
mpl.rcParams['savefig.facecolor'] = 'w'
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import csky as cy
from csky import cext
import numpy as np
import astropy
import histlite as hl
import healpy
import healpy as hp
import socket
import pickle
from scipy import stats
import copy
healpy.disable_warnings()
plt.rc('figure', facecolor = 'w')
plt.rc('figure', dpi=100)

## Define Settings

In [2]:
selection_version = 'version-001-p01'

host_name = socket.gethostname()

if 'cobalt' in host_name:
    print('Working on Cobalts')
    #data_prefix = '/data/user/ssclafani/data/cscd/final'
    ana_dir = '/data/user/ssclafani/data/analyses/'
    plot_dir = cy.utils.ensure_dir('/data/user/ssclafani/data/analyses/DNNCascadeCodeReview/unblinding_checks/plots/unblinding/galactic_plane_checks')
    
else:
    raise ValueError('Unknown host:', host_name)

Working on Cobalts


In [3]:
for dir_path in [plot_dir]:
    if not os.path.exists(dir_path):
        print('Creating directory:', dir_path)
        os.makedirs(dir_path)

## Load Data

In [4]:
repo = cy.selections.Repository(local_root='/data/user/ssclafani/data/analyses')
specs = cy.selections.DNNCascadeDataSpecs.DNNC_10yr

In [5]:
%%time

ana = cy.get_analysis(
    repo, selection_version, specs, 
    #gammas=np.r_[0.1:6.01:0.125],
)

Setting up Analysis for:
DNNCascade_10yr
Setting up DNNCascade_10yr...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p01/MC_NuGen_bfrv1_2153x.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p01/IC86_2011_exp.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p01/IC86_2012_exp.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p01/IC86_2013_exp.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p01/IC86_2014_exp.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p01/IC86_2015_exp.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p01/IC86_2016_exp.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p01/IC86_2017_exp.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p01/IC86_2018_exp.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p01/IC86

## Setup Analysis

In [6]:
import sys
sys.path.insert(0, '../..')

import config as cg

cg.base_dir = '/data/user/mhuennefeld/data/analyses/unblinding_v1.0.1_csky_bugfix_template_flux/'

Running as User: ssclafani on Hostname: cobalt07.icecube.wisc.edu


In [7]:
import pandas as pd

In [8]:
source_list = pd.read_pickle('/home/ssclafani/DNNCascade/catalogs/Source_List_DNNC.pickle')

In [9]:
src_list_file = os.path.join(cg.catalog_dir, 'Source_List_DNNC.npy')
sourcelist = np.load(src_list_file, allow_pickle=True)

# load correlated MultiTrialRunner background trials
print('Loading correlated background trials...')
base_dir = os.path.join('/data/user/mhuennefeld/data/analyses/unblinding_v1.0.1/ps/correlated_trials/')
bgfile_corr = '{}/correlated_bg.npy'.format(base_dir)
trials_corr = np.load(bgfile_corr, allow_pickle=True)
bg_corr = cy.dists.TSD(trials_corr)

# load bkg trials at source declinations
print('Loading background trials at source declinations...')
bgfile = '{}/pretrial_bgs.npy'.format(base_dir)
bgs = np.load(bgfile, allow_pickle=True)                             


Loading correlated background trials...
Loading background trials at source declinations...


In [10]:
base_dir = '/data/user/mhuennefeld/data/analyses/unblinding_v1.0.1/'
results = np.load(base_dir + '/ps/results/fulllist_unblinded.npy', allow_pickle=True)

In [11]:
sig = np.load('/data/user/ssclafani/data/analyses/baseline_unblind_v1.0.0/ps/correlated_trials/sourcelist_sig.npy' , allow_pickle=True)

In [12]:

def get_tr(dec, ra,  gamma):
    src = cy.utils.sources(ra, dec, deg=False)                  
    conf = cg.get_ps_conf(src=src, gamma=gamma)
    tr = cy.get_trial_runner(ana=ana, conf=conf, TRUTH=False)
    return tr, src
def get_ul(gamma, verbose=False):
    uls90 = []
    for i, source in enumerate(sourcelist):
        tr, src = get_tr(
        dec=np.radians(source[2]),
        ra=np.radians(source[1]),
        gamma=gamma)
        ts = results[0][i][0]
        bg = bgs[i]
        trials = sig[i]['gamma'][gamma]['cutoff_TeV'][np.inf]['nsig']

        trials[0] = bg.trials
        print('Source {}: Getting fraction of trials > {:.4}: For Gamma = {}'.format(i, ts, gamma))
        ul90 = tr.find_n_sig(ts, 0.9, trials=trials, logging=verbose, tol=.1)
        ul90['flux'] = tr.to_E2dNdE(ul90['n_sig'], E0=100, unit=1e3)
        uls90.append(ul90['flux'])
        print(ul90['flux'], ul90['n_sig'])
    return uls90

In [13]:
uls90_E2 = get_ul(2.0, verbose=False)

Source 0: Getting fraction of trials > 8.922: For Gamma = 2.0
3.3449307915003384e-12 23.036590452977702
Source 1: Getting fraction of trials > 0.00193: For Gamma = 2.0
5.89059800493371e-13 3.9923470727150874
Source 2: Getting fraction of trials > 0.08578: For Gamma = 2.0
8.088054140662726e-13 5.43699768652717
Source 3: Getting fraction of trials > 1.743: For Gamma = 2.0
1.5160457557781774e-12 10.661087631449055
Source 4: Getting fraction of trials > 0.0: For Gamma = 2.0
1.126666008895705e-12 7.461785878369955
Source 5: Getting fraction of trials > 0.1957: For Gamma = 2.0
8.940588469555712e-13 6.153978219290559
Source 6: Getting fraction of trials > 0.01047: For Gamma = 2.0
6.214343189172727e-13 4.274443423544855
Source 7: Getting fraction of trials > 0.8315: For Gamma = 2.0
2.72969979939104e-12 17.952538458722533
Source 8: Getting fraction of trials > 0.0: For Gamma = 2.0
6.822935045775538e-13 4.96992970059299
Source 9: Getting fraction of trials > 5.313: For Gamma = 2.0
4.243457690034

In [14]:
np.save('sourcelist_UL90_E2_patch_01.npy' ,uls90_E2)

In [15]:
uls90_E3 = get_ul(3.0, verbose=False)

Source 0: Getting fraction of trials > 8.922: For Gamma = 3.0
1.577585008081677e-12 96.65842524202417
Source 1: Getting fraction of trials > 0.00193: For Gamma = 3.0
2.7910791094841396e-13 16.98889204954507
Source 2: Getting fraction of trials > 0.08578: For Gamma = 3.0
4.152196734180769e-13 26.142976192022985
Source 3: Getting fraction of trials > 1.743: For Gamma = 3.0
1.0170710802417905e-12 64.51409775583056
Source 4: Getting fraction of trials > 0.0: For Gamma = 3.0
3.1902309524399886e-13 36.53927054012679
Source 5: Getting fraction of trials > 0.1957: For Gamma = 3.0
4.541896040882579e-13 27.884836181478644
Source 6: Getting fraction of trials > 0.01047: For Gamma = 3.0
3.062215184087817e-13 18.84393774359547
Source 7: Getting fraction of trials > 0.8315: For Gamma = 3.0
7.651500759742972e-13 88.82671932135152
Source 8: Getting fraction of trials > 0.0: For Gamma = 3.0
3.535517210395227e-13 33.35247630305181
Source 9: Getting fraction of trials > 5.313: For Gamma = 3.0
1.593613408

In [16]:
np.save('sourcelist_UL90_E3_patch_01.npy' ,uls90_E3)

Running as User: ssclafani on Hostname: cobalt07.icecube.wisc.edu
